## ThirdAI's NeuralDB

First let's import the relevant module and initialize a neural db class.

In [4]:
from thirdai import neural_db as ndb

db = ndb.NeuralDB(user_id="my_user")

### Initialize
At this point, the db is uninitialized. We can either initialize from scratch like this

In [18]:
db.from_scratch()

Or load a checkpoint / base model as follows:

In [ ]:
db.from_checkpoint("path/to/checkpoint")

### Insert

Let's insert things into it!

Currently, we support adding as many CSV files as you wish. All other file formats have to be converted into CSV files where each row represents a paragraph/text-chunk of the document. 

The file is required to have a column named "DOC_ID" with rows numbered from 0 to n_rows-1.

In [ ]:
from utils import CSVDocument

csv_doc = CSVDocument(
    path="sample_nda.csv",
    strong_columns=["passage"],
    weak_columns=["para"],  
    reference_columns=["passage"])

# Just like that!
db.insert([csv_doc])

In [ ]:
sup_files = ['sample_nda_train.csv']

db.supervised_train([ndb.Sup(file, query_column="QUERY", id_column="DOC_ID") for file in sup_files])

### Search

That's it! Now let's start searching.

In [20]:
search_results = db.search(
    query="what is the termination period",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text())
    # print(result.source())
    # print(result.metadata())
    # result.show()

12. entire agreement. this agreement constitutes the entire agreement with respect to the subject matter hereof and supersedes all prior agreements and understandings between the parties (whether written or oral) relating to the subject matter and may not be amended or modified except in a writing signed by an authorized representative of both parties. the terms of this agreement relating to the confidentiality and non-use of confidential information shall continue after the termination of this agreement for a period of the longer of (i) five (5) years or (ii) when the confidential information no longer qualifies as a trade secret under applicable law.
13. severability. each party acknowledges that should any provision of this agreement be determined to be void invalid or otherwise unenforceable by any court of competent jurisdiction such determination shall not affect the remaining provisions hereof which shall remain in full force and effect.


We can see that the search pulled up the right passage that contains the termination period "(i) five (5) years or (ii) when the confidential information no longer qualifies as a trade secret" .

Now let's ask a slightly trickier question.

In [21]:
search_results = db.search(
    query="who are the parties involved?",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text())
    # print(result.source())
    # print(result.metadata())
    # result.show()

3. joint undertaking. each party agrees that it will not at any time disclose give or transmit in any manner or for any purpose the confidential information received from the other party to any person firm or corporation or use such confidential information for its own benefit or the benefit of anyone else or for any purpose other than to engage in discussions regarding a possible business relationship or the current business relationship involving both parties.
in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023


Oops! looks like when we search for "parties involved", we get the correct paragraph in the 2nd position but not in the first place. 

No worries, we'll show shot to teach the model to correct it's retrieval.

### RLHF

Let's go over some of NeuralDB's advanced features. The first one is text-to-text association. This allows you to teach the model that two keywords, phrases, or concepts are related.

Based on the above example, let's teach the model that "parties involved" and the phrase "made by between" are the same.

In [26]:
db.associate(source="parties involved", target="made by and between")

Let's search again with the same query.

In [28]:
search_results = db.search(
    query="who are the parties involved?",
    top_k=2,
)

for result in search_results:
    print(result.text())
    # print(result.source())
    # print(result.metadata())
    # result.show()

confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).
in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023


There you go! In just a line, you taught the model to correct itself and retrieve the correct result. 

### Load and Save
As usual, saving and loading are one-liners.

In [ ]:
# save your db
db.save("path/to/checkpoint/")

# Loading is just like we showed above, with an optional progress handler
db.from_checkpoint("path/to/checkpoint", on_progress=lambda fraction: print(f"{fraction}% done with loading."))

### Export to Playground